In [1]:
import cv2
import mediapipe as mp
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time
import numpy as np

pygame 2.3.0 (SDL 2.24.2, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
cap=cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2

In [3]:
mixer.init()
sound = mixer.Sound('alarm.wav')

In [4]:
mpDraw=mp.solutions.drawing_utils
mpFaceMesh=mp.solutions.face_mesh
facemesh=mpFaceMesh.FaceMesh(max_num_faces=1)
drawSpec=mpDraw.DrawingSpec(thickness=1,circle_radius=2)

In [5]:
model2=load_model("cnnCat2.h5")
path = os.getcwd()

In [6]:
def getRightEye(image, landmarks):
    eye_top = int(landmarks[443].y * image.shape[0])
    eye_left = int(landmarks[464].x * image.shape[1])
    eye_bottom = int(landmarks[450].y * image.shape[0])
    eye_right = int(landmarks[446].x * image.shape[1])
    right_eye = image[eye_top:eye_bottom, eye_left:eye_right]
    return right_eye

In [7]:
def getRightEyeRect(image, landmarks):
    eye_top = int(landmarks[443].y * image.shape[0])
    eye_left = int(landmarks[464].x * image.shape[1])
    eye_bottom = int(landmarks[450].y * image.shape[0])
    eye_right = int(landmarks[446].x * image.shape[1])

    cloned_image = image.copy()
    cropped_right_eye = cloned_image[eye_top:eye_bottom, eye_left:eye_right]
    h, w, _ = cropped_right_eye.shape
    x = eye_left
    y = eye_top
    return x, y, w, h

In [8]:
def getLeftEye(image, landmarks):
    eye_top = int(landmarks[223].y * image.shape[0])
    eye_left = int(landmarks[226].x * image.shape[1])
    eye_bottom = int(landmarks[230].y * image.shape[0])
    eye_right = int(landmarks[243].x * image.shape[1])
    left_eye = image[eye_top:eye_bottom, eye_left:eye_right]
    return left_eye


def getLeftEyeRect(image, landmarks):
    # eye_left landmarks (27, 23, 130, 133) ->? how to utilize z info
    eye_top = int(landmarks[223].y * image.shape[0])
    eye_left = int(landmarks[226].x * image.shape[1])
    eye_bottom = int(landmarks[230].y * image.shape[0])
    eye_right = int(landmarks[243].x * image.shape[1])

    cloned_image = image.copy()
    cropped_left_eye = cloned_image[eye_top:eye_bottom, eye_left:eye_right]
    h, w, _ = cropped_left_eye.shape

    x = eye_left
    y = eye_top
    return x, y, w, h

In [9]:
def getMouth(image, landmarks):
    mouth_top = int(landmarks[164].y * image.shape[0])
    mouth_left = int(landmarks[57].x * image.shape[1])
    mouth_bottom = int(landmarks[18].y * image.shape[0])
    mouth_right = int(landmarks[287].x * image.shape[1])
    mouth = image[mouth_top:mouth_bottom, mouth_left:mouth_right]
    return mouth


def getMouthRect(image, landmarks):
    # eye_left landmarks (27, 23, 130, 133) ->? how to utilize z info
    mouth_top = int(landmarks[164].y * image.shape[0])
    mouth_left = int(landmarks[57].x * image.shape[1])
    mouth_bottom = int(landmarks[18].y * image.shape[0])
    mouth_right = int(landmarks[287].x * image.shape[1])

    cloned_image = image.copy()
    cropped_mouth = cloned_image[mouth_top:mouth_bottom, mouth_left:mouth_right]
    h, w, _ = cropped_mouth.shape

    x = mouth_left
    y = mouth_top
    return x, y, w, h

In [10]:
lbl=['Closed_eye','Open_eye',"Yawn","No_Yawn"]

In [11]:
while True:
    success,img=cap.read()
    height,width=img.shape[:2]
    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=facemesh.process(imgRGB)
    if results.multi_face_landmarks:
        for faceLand_Marks in results.multi_face_landmarks:
            #mpDraw.draw_landmarks(img,faceLand_Marks,mpFaceMesh.FACEMESH_LIPS,drawSpec,drawSpec)
            right_eye_img=getRightEye(img,faceLand_Marks.landmark)
            right_eye_height,right_eye_width,_=right_eye_img.shape
            xRightEye,yRightEye,rightEyeWidth,rightEyeHeight=getRightEyeRect(img,faceLand_Marks.landmark)
            cv2.rectangle(img,(xRightEye,yRightEye),(xRightEye+right_eye_width,yRightEye+right_eye_height),(200,0,0),2)
            
            left_eye_img=getLeftEye(img,faceLand_Marks.landmark)
            left_eye_height,left_eye_width,_=left_eye_img.shape
            xLeftEye,yLeftEye,leftEyeWidth,leftEyeHeight=getLeftEyeRect(img,faceLand_Marks.landmark)
            cv2.rectangle(img,(xLeftEye,yLeftEye),(xLeftEye+left_eye_width,yLeftEye+left_eye_height),(200,0,0),2)
            
            mouth_img=getMouth(img,faceLand_Marks.landmark)
            mouth_height,mouth_width,_=mouth_img.shape
            xMouth,yMouth,MouthWidth,MouthHeight=getMouthRect(img,faceLand_Marks.landmark)
            cv2.rectangle(img,(xMouth,yMouth),(xMouth+mouth_width,yMouth+mouth_height),(0,0,255),2)
            
            
            right_eye=img[yRightEye:yRightEye+right_eye_height,xRightEye:xRightEye+right_eye_width]
            
            left_eye=img[yLeftEye:yLeftEye+left_eye_height,xLeftEye:xLeftEye+left_eye_width]
            
            mouth=img[yMouth:yMouth+mouth_height,xMouth:xMouth+mouth_width]
            
            mouth=cv2.cvtColor(mouth,cv2.COLOR_BGR2GRAY)
            mouth=cv2.resize(mouth,(24,24))
            mouth=mouth/255
            mouth=mouth.reshape(24,24,-1)
            mouth=np.expand_dims(mouth,axis=0)
            mouthpred=model2.predict(mouth)
            #print(mouthpred)
            mouthpred=np.argmax(mouthpred,axis=1)
            if mouthpred[0]==4:
                #print("Yawn")
                lbl="Yawn"
            elif mouthpred[0]==3:
                #print("No_Yawn")
                lbl="No_Yawn"
            
            
            right_eye=cv2.cvtColor(right_eye,cv2.COLOR_BGR2GRAY)
            right_eye=cv2.resize(right_eye,(24,24))
            right_eye=right_eye/255
            right_eye=right_eye.reshape(24,24,-1)
            right_eye=np.expand_dims(right_eye,axis=0)
            rpred=model2.predict(right_eye)
            print(rpred)
            rpred=np.argmax(rpred,axis=1)
            if rpred[0]==2:
                lbl="Open_eye"
                print(lbl)
            elif rpred[0]==1:
                print(lbl)
                lbl="Closed_eye"
                
            
            left_eye=cv2.cvtColor(left_eye,cv2.COLOR_BGR2GRAY)
            left_eye=cv2.resize(left_eye,(24,24))
            left_eye=left_eye/255
            left_eye=left_eye.reshape(24,24,-1)
            left_eye=np.expand_dims(left_eye,axis=0)
            lpred=model2.predict(left_eye)
            lpred=np.argmax(lpred,axis=1)
            if lpred[0]==2:
                lbl="Open_eye"
            elif lpred[0]==1:
                lbl="Close_eye"
            
            
            if (rpred[0]==1 or lpred[0]==1 or mouthpred[0]==4 ):
                score=score+1
                cv2.putText(img,"Closed or Yawning",(300,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
                # if(rpred[0]==1 or lpred[0]==1):
            else:
                score=score-1
                cv2.putText(img,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
                
            if(score<0):
                score=0   
            cv2.putText(img,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            if(score>10):
                #person is feeling sleepy so we beep the alarm
                cv2.imwrite(os.path.join(path,'image.jpg'),img)
                try:
                    sound.play()
                    j
                except:  # isplaying = False
                    pass
                if(thicc<16):
                    thicc= thicc+2
                else:
                    thicc=thicc-2
                    if(thicc<2):
                        thicc=2
                cv2.rectangle(img,(0,0),(width,height),(0,0,255),thicc) 
            

            
            
    
    
    
    
    cv2.imshow("Image",img)
    if(cv2.waitKey(20) & 0XFF==ord("d")):
        break

1/1 [==============================] - 0s 22ms/step
[[1.9212870e-04 7.3711909e-02 8.7768114e-01 3.4041949e-02 1.4372797e-02]]
Open_eye
1/1 [==============================] - 0s 23ms/step
[[1.3314946e-04 8.0703728e-02 1.3097462e-01 7.7347267e-01 1.4715837e-02]]
1/1 [==============================] - 0s 21ms/step
[[2.59397901e-04 1.24047965e-01 1.70013085e-01 6.80447519e-01
  2.52320599e-02]]
1/1 [==============================] - 0s 20ms/step
[[2.6504498e-04 1.2797253e-01 5.0457459e-02 7.9753643e-01 2.3768486e-02]]
1/1 [==============================] - 0s 21ms/step
[[2.5246170e-04 1.0462195e-01 4.3353267e-02 8.2622653e-01 2.5545776e-02]]
1/1 [==============================] - 0s 21ms/step
[[4.8179645e-04 9.9770471e-02 1.6538547e-01 6.9210148e-01 4.2260811e-02]]
1/1 [==============================] - 0s 20ms/step
[[2.0120241e-04 3.9805885e-02 7.9695392e-01 1.4143935e-01 2.1599563e-02]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[0.00052458 0.1236772  0.4333072  0.4070

1/1 [==============================] - 0s 21ms/step
[[4.6100515e-05 2.9433075e-02 9.1130561e-01 5.1877778e-02 7.3374668e-03]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[6.6260181e-05 3.7530646e-02 9.0253955e-01 5.0332543e-02 9.5309308e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[3.6317178e-05 1.2887192e-02 9.5353276e-01 2.7648641e-02 5.8951834e-03]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[2.1962001e-05 8.1124380e-03 9.8148674e-01 7.1358043e-03 3.2430727e-03]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[3.8807106e-05 3.5838950e-02 9.1799289e-01 4.0310789e-02 5.8185719e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[6.0445444e-05 5.5417418e-02 8.5424531e-01 8.1133910e-02 9.1429763e-03]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[6.7012443e-05 3.5475373e-02 8.2676685e-01 1.2544152e-01 1.2249226e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[0

1/1 [==============================] - 0s 21ms/step
[[1.1875507e-04 1.4465562e-01 3.3943677e-01 5.0236136e-01 1.3427530e-02]]
1/1 [==============================] - 0s 20ms/step
[[1.5649146e-04 2.0980048e-01 4.0808624e-01 3.6509672e-01 1.6860066e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[3.6357448e-04 2.0562203e-01 5.8649307e-01 1.8078928e-01 2.6732063e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[1.4720189e-04 1.4240819e-01 5.6997532e-01 2.7121297e-01 1.6256312e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[1.9602486e-04 1.0709788e-01 5.4591173e-01 3.2647213e-01 2.0322314e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[0.00050327 0.29009274 0.46839914 0.21013854 0.03086629]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[9.2778442e-05 1.0645353e-01 7.8316396e-01 9.9439234e-02 1.0850572e-02]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[1.24426588e-04 1.00048013

1/1 [==============================] - 0s 20ms/step
[[8.6336193e-05 1.3276494e-01 8.2495010e-01 3.4387190e-02 7.8114420e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[7.2680938e-05 1.7347051e-01 7.9609561e-01 2.3236662e-02 7.1245604e-03]]
Open_eye
1/1 [==============================] - 0s 22ms/step
[[1.8207413e-04 7.4011874e-01 1.9468752e-01 5.4678243e-02 1.0333402e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[1.10422174e-04 5.67977190e-01 3.64918351e-01 5.80375195e-02
  8.95651896e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[1.1780184e-04 4.8719946e-01 4.1285765e-01 9.0164535e-02 9.6605979e-03]]
Open_eye
1/1 [==============================] - 0s 22ms/step
[[1.5064856e-04 5.2747029e-01 3.7059927e-01 9.0479434e-02 1.1300425e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[1.5802972e-04 6.6668326e-01 2.6724723e-01 5.6580577e-02 9.3308715e-03]]
Open_eye
1/1 [==============================] - 0s 20ms/s

1/1 [==============================] - 0s 21ms/step
[[3.2956756e-04 1.5875538e-01 6.8394780e-01 1.3422583e-01 2.2741441e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[3.1113257e-06 1.8189719e-03 9.9688655e-01 4.1240663e-04 8.7906781e-04]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[8.8411534e-06 6.4797639e-03 9.8952466e-01 2.1980922e-03 1.7886843e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[8.1280623e-06 8.2343034e-03 9.8772591e-01 2.1879170e-03 1.8437175e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[6.8069999e-06 8.0093127e-03 9.8614562e-01 4.0402901e-03 1.7979763e-03]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[4.2305044e-05 2.0678062e-02 9.6814871e-01 6.4343014e-03 4.6966393e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[8.6012533e-06 1.5684692e-02 9.8075014e-01 2.0837551e-03 1.4728040e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[3

...

1/1 [==============================] - 0s 21ms/step
[[7.4165570e-07 3.7871598e-04 3.8978622e-05 8.6615866e-01 1.3342299e-01]]
1/1 [==============================] - 0s 21ms/step
[[1.0252528e-06 3.0471972e-04 5.7165664e-05 9.2677873e-01 7.2858281e-02]]
1/1 [==============================] - 0s 21ms/step
[[2.1600628e-05 1.8774220e-03 5.5569893e-04 8.2802421e-01 1.6952117e-01]]
1/1 [==============================] - 0s 21ms/step
[[2.6788980e-06 4.9890095e-04 1.6579777e-04 6.8987644e-01 3.0945617e-01]]
1/1 [==============================] - 0s 21ms/step
[[1.6111700e-05 1.8218859e-03 2.6687022e-04 6.8796420e-01 3.0993101e-01]]
1/1 [==============================] - 0s 20ms/step
[[0.00153963 0.03514409 0.01606597 0.5749268  0.3723235 ]]
1/1 [==============================] - 0s 20ms/step
[[1.1227213e-06 3.0580407e-04 9.2908813e-05 8.9984566e-01 9.9754572e-02]]
1/1 [==============================] - 0s 20ms/step
[[6.8691446e-07 2.0386366e-04 6.3690626e-05 9.5667154e-01 4.3060228e-02]]
1/1 [==

1/1 [==============================] - 0s 21ms/step
[[9.6124386e-05 6.5397537e-01 2.7128562e-01 6.5376908e-02 9.2660105e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[1.3878933e-04 1.6405404e-01 7.6191550e-01 5.9770018e-02 1.4121729e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[1.3069084e-04 4.2732584e-01 4.9660507e-01 6.2638178e-02 1.3300211e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[5.1088668e-05 1.0354908e-01 8.6275762e-01 2.6591441e-02 7.0508383e-03]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[1.1302939e-04 4.3429312e-01 5.3696465e-01 2.0506680e-02 8.1224348e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[4.7214115e-05 6.8136394e-01 3.0777147e-01 6.4015477e-03 4.4158236e-03]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[4.7393882e-05 7.9067481e-01 1.9594158e-01 9.4618378e-03 3.8744083e-03]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[9

[[4.0342048e-04 4.2867619e-01 2.6430666e-01 2.7751976e-01 2.9094128e-02]]
No_Yawn
1/1 [==============================] - 0s 21ms/step
[[0.00039688 0.38707098 0.35000232 0.23835462 0.02417519]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[2.4916546e-04 4.4678876e-01 4.3643236e-01 9.5508091e-02 2.1021605e-02]]
Closed_eye
1/1 [==============================] - 0s 20ms/step
[[2.5840881e-04 5.8002454e-01 2.0640396e-01 1.9516869e-01 1.8144531e-02]]
No_Yawn
1/1 [==============================] - 0s 22ms/step
[[3.4825780e-04 5.7053119e-01 1.8723680e-01 2.2246318e-01 1.9420596e-02]]
No_Yawn
1/1 [==============================] - 0s 21ms/step
[[2.9824636e-04 3.1252697e-01 6.2248957e-01 4.2964019e-02 2.1721167e-02]]
Open_eye
1/1 [==============================] - 0s 21ms/step
[[1.9841139e-07 9.9960631e-01 2.3441436e-04 5.5083383e-05 1.0404693e-04]]
Open_eye
1/1 [==============================] - 0s 20ms/step
[[0.00048016 0.32611278 0.16854008 0.47173056 0.03313654]]
1/1 [=======

1/1 [==============================] - 0s 23ms/step
[[2.7655161e-04 8.9359000e-02 8.1760550e-01 7.4427143e-02 1.8331788e-02]]
Open_eye
1/1 [==============================] - 0s 22ms/step
[[4.39586875e-04 1.25720739e-01 7.37597883e-01 1.10882774e-01
  2.53589526e-02]]
Open_eye
1/1 [==============================] - 0s 22ms/step
[[2.1331882e-04 1.4524302e-01 6.1430115e-01 2.2299729e-01 1.7245140e-02]]
Open_eye
1/1 [==============================] - 0s 30ms/step
[[3.2056391e-04 2.1002036e-01 6.4256078e-01 1.2598571e-01 2.1112572e-02]]
Open_eye
1/1 [==============================] - 0s 23ms/step
[[6.1072613e-05 9.2255734e-03 9.4998664e-01 3.1346198e-02 9.3805874e-03]]
Open_eye
1/1 [==============================] - 0s 22ms/step
[[2.6653698e-04 4.6597283e-02 6.2050277e-01 2.9513028e-01 3.7503161e-02]]
Open_eye
1/1 [==============================] - 0s 27ms/step
[[1.63514807e-04 3.77562940e-02 9.37470913e-01 1.17606325e-02
  1.28486436e-02]]
Open_eye
1/1 [==============================] - 0s

KeyboardInterrupt: 

In [12]:
cap.release()
cv2.destroyAllWindows()